In [83]:
import pandas as pd
import numpy as np

# Procesamiento columna 'Descripción'

In [84]:
excel_file = pd.ExcelFile('../data/raw/data.xlsx')
sheets = excel_file.sheet_names
sheets

['Lista de equipos',
 'Historico calif 2_1(10 años)',
 'T-PAP(C)_T-ACE',
 'ARC(I)_ARC(D)',
 'DP',
 'OIL.TAP',
 'DIEL.PF_ DIEL',
 'OIL.CORR',
 'PRO-M',
 'M.OLTC']

In [85]:
import unidecode

def norm_descrpcion(s):
    s = s.lower()
    s = unidecode.unidecode(s)
    return s

converter_descripcion = {}
converter_descripcion['Descripción'] = norm_descrpcion

df_collection = {}
for name in sheets:
    df_collection[name] = pd.read_excel('../data/raw/data.xlsx', sheet_name=name, converters=converter_descripcion)

1. Separar cada fila en cada una de las descripciones. i.e : "realizar analisis dga, humedad y furanos" deben quedar dos filas exactamente iguales pero con cada uno de los registros. -> pd.explode(), explode a esa accion de expandir un dataset con base a una regla.

2. Aplicar un algoritmo de distancia de levehnstein. 

3. Análisis descriptivo que relacione cada equipo con las acciones. i.e: frecuencias de cada accion para cada equipo.

## Separación manual

In [86]:
def separacion_manual(dframe, originales, reemplazos, adiciones):
    df = dframe.copy()
    for i in range(len(originales)):
        index = df['Descripción'] == originales[i]
        if sum(index) > 0:
            instancias = df.loc[index].copy()
            df.loc[index,'Descripción'] = reemplazos[i]
            df = df.append(instancias, ignore_index = True)
            df.loc[len(df) - sum(index):,'Descripción'] = adiciones[i]
    return df




df_collection_separados = {}

excel_file_reemplazos = pd.ExcelFile('../data/reemplazos_manual/reemplazos_manual.xlsx')
sheets_reemplazos = excel_file_reemplazos.sheet_names
df_collection_manuales = {}

converter_manuales = {}
converter_manuales['Originales'] = norm_descrpcion
converter_manuales['Reemplazos'] = norm_descrpcion
converter_manuales['Adiciones'] = norm_descrpcion

for name in sheets_reemplazos:#sheets_reemplazos:
    df_collection_manuales[name] = pd.read_excel('../data/reemplazos_manual/reemplazos_manual.xlsx', sheet_name = name, converters = converter_manuales)
    index_manuales = ~df_collection_manuales[name].loc[:,'Reemplazos'].isna()
    originales = df_collection_manuales[name].loc[index_manuales, 'Originales'].values
    reemplazos = df_collection_manuales[name].loc[index_manuales, 'Reemplazos'].values
    adiciones = df_collection_manuales[name].loc[index_manuales, 'Adiciones'].values
    df_collection_separados[name] = separacion_manual(df_collection[name], originales, reemplazos, adiciones)

## Aplicando el algoritmo de distancia de Levenshtein

### Algoritmo de Levenshtein

In [87]:
def dist_lev(s1,s2):
    dist_matrix = np.zeros((len(s1) + 1, len(s2) + 1))
    dist_matrix[1:,0] = range(1,len(s1) + 1)
    dist_matrix[0,1:] = range(1,len(s2) + 1)
    for i in range(1, len(s1) + 1):
        for j in range(1, len(s2) + 1):
            if s1[i - 1] == s2[j - 1]:
                dist_matrix[i,j] = min(dist_matrix[i - 1, j] + 1, 
                                    dist_matrix[i - 1, j - 1], 
                                    dist_matrix[i, j - 1] + 1)
            else:
                dist_matrix[i,j] = min(dist_matrix[i - 1, j] + 1, 
                                    dist_matrix[i - 1, j - 1] + 1, 
                                    dist_matrix[i, j - 1] + 1)
    return dist_matrix[-1,-1]

### Aplicando y ordenando según distancia

In [88]:
# Extrayendo valores únicos:
ordenados_collection = {}
dist_ordenados_collection = {}
index_ordenados_collection = {}
for name in sheets_reemplazos:
    unicos = pd.unique(df_collection_separados[name]['Descripción'])
    #Construyendo matriz de distancia de Levenshtein
    dist_unicos = np.zeros((len(unicos), len(unicos)))
    for i in range(len(unicos)):
        for j in range(i + 1, len(unicos)):
            dist_unicos[i, j] = dist_lev(unicos[i], unicos[j]) 
            dist_unicos[j, i] = dist_unicos[i, j]  
    #Construyendo array de valores únicos ordenados
    #Inizializar varibales
    ordenados = ['']*len(unicos)
    dist_ordenados = np.zeros(len(unicos))
    index_ordenados = np.zeros(len(unicos))
    #Ubicando primeros dos elementos
    max_dist_index = np.argmax(dist_unicos)
    max_dist_index = np.unravel_index(max_dist_index, dist_unicos.shape)

    ordenados[0] = unicos[max_dist_index[0]]
    dist_ordenados[0] = 0
    index_ordenados[0] = max_dist_index[0]
            
    ordenados[-1] = unicos[max_dist_index[1]]
    dist_ordenados[-1] = dist_unicos[max_dist_index]
    index_ordenados[-1] = max_dist_index[1]

    dist_unicos[max_dist_index] = np.inf
    dist_unicos[max_dist_index[1], max_dist_index[0]] = np.inf
    #Extrayendo la fila que contiene a la distancia máxima
    row_index = max_dist_index[0]
    row = dist_unicos[row_index,:]
    row[row_index] = np.inf #Elemento de la diagonal
    #Ubicando el resto de los elementos
    for i in range(1, len(unicos) - 1):
        min_index = np.argmin(row)
        ordenados[i] = unicos[min_index]
        dist_ordenados[i] = dist_unicos[row_index, min_index]
        index_ordenados[i] = min_index
        dist_unicos[row_index,min_index] = np.inf   

    #Ubicándolos por hoja
    ordenados_collection[name] = ordenados
    dist_ordenados_collection[name] = dist_ordenados
    index_ordenados_collection[name] = index_ordenados


### Escribiendo los resultados en Excel

In [89]:
import xlsxwriter
workbook = xlsxwriter.Workbook('../data/reemplazos_manual/unicos_ordenados.xlsx')
for name in sheets_reemplazos:
    worksheet = workbook.add_worksheet(name)
    for i in range(len(ordenados_collection[name])): 
        worksheet.write(i,0,ordenados_collection[name][i])
        worksheet.write(i,1,ordenados_collection[name][i])
        worksheet.write(i,2,dist_ordenados_collection[name][i])
        worksheet.write(i,3,index_ordenados_collection[name][i])

workbook.close()

In [90]:
len(ordenados_collection[name])
# workbook.close()

94